In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name

In [2]:
tf_version = '1.15.0'

In [3]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/ mnist --recursive

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to mnist/eval_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_labels.npy to mnist/train_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to mnist/eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_data.npy to mnist/train_data.npy


In [4]:
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [17]:
# mnist 훈련 스크립트
training_script = "../mnist.py"

In [18]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # MNIST 훈련 script
    entry_point=training_script,
    role=role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    framework_version=tf_version,
    py_version="py3",
)

In [19]:
# Sagemaker 훈련 작업 시작
estimator.fit(training_data_uri)

2021-11-17 11:10:11 Starting - Starting the training job...
2021-11-17 11:10:37 Starting - Launching requested ML instancesProfilerReport-1637147411: InProgress
......
2021-11-17 11:11:37 Starting - Preparing the instances for training.........
2021-11-17 11:13:07 Downloading - Downloading input data
2021-11-17 11:13:07 Training - Downloading the training image.........
2021-11-17 11:14:38 Training - Training image download completed. Training in progress.WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1467: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
2021-11-17 11:14:28,095 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-11-17 11:14:28,545 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host

In [20]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_data.npy eval_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_labels.npy eval_labels.npy

eval_data = np.load("eval_data.npy")
eval_labels = np.load("eval_labels.npy")

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to ./eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to ./eval_labels.npy


In [21]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [26]:
import time

start = time.time()
for i in range(0, 10000):
    data = eval_data[i]
    predictions = predictor.predict(data)
print("time :", time.time() - start)

time : 83.92276453971863


In [31]:
predictor.delete_endpoint()

In [27]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_c5",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path,
    framework="tensorflow",
    framework_version="1.15.0",
)

??????????????????????????.................................!

In [28]:
# 모델 최적화 후 실시간으로 예측 요청을 제공하는 Endpoint에 배포
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.c5.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [30]:
import time

start = time.time()

for i in range(0, 10000):
    data = eval_data[i]
    predictions = optimized_predictor.predict(data)
print("time :", time.time() - start)

time : 70.73337697982788


In [32]:
optimized_predictor.delete_endpoint()